<a href="https://colab.research.google.com/github/YNNJN/colorPool_Data/blob/master/%EA%B8%B0%EC%A4%80%ED%96%89%EB%A0%AC%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/YNNJN/colorPool_Data.git

In [ ]:
!ls -ltr

In [ ]:
# connect to google dirve

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

colorData = pd.read_csv('gdrive/My Drive/colorPOOL/kulerData/quantize.csv')
colorData = colorData.drop([colorData.columns[0]], axis=1)
colorData

In [ ]:
# 데이터프레임 형태 변환 15000*3 => 3000*15

df1=colorData.loc[:3000].reset_index(drop=True).rename(columns={"L":'L1',"a":'a1',"b":'b1'})
df2=colorData.loc[3000:6000].reset_index(drop=True).rename(columns={"L":'L2',"a":'a2',"b":'b2'})
df3=colorData.loc[6000:9000].reset_index(drop=True).rename(columns={"L":'L3',"a":'a3',"b":'b3'})
df4=colorData.loc[9000:12000].reset_index(drop=True).rename(columns={"L":'L4',"a":'a4',"b":'b4'})
df5=colorData.loc[12000:15000].reset_index(drop=True).rename(columns={"L":'L5',"a":'a5',"b":'b5'})

df_all=pd.concat([df1,df2,df3,df4,df5], axis=1).reset_index(drop=True)
df_all

df_all.to_csv('gdrive/My Drive/colorPOOL/kulerData/theme&color.csv')